In [ ]:
from pathlib import Path
from loguru import logger
from typing import Any
import pandas as pd

In [71]:
signatures_path = Path("../data/sigs.csv")
df = pd.read_csv(signatures_path)
logger.info("loaded signatures data: {} rows {} columns", df.shape[0], df.shape[1])
old_name = df.columns[0]
df.rename(columns={old_name:"cell_id"}, inplace=True)

2024-06-14 18:59:17.644 | INFO     | __main__:<module>:3 - loaded signatures data: 9370 rows 3 columns


Index(['cell_id', 'cluster', 'signature'], dtype='object')

In [72]:

train_df = df.sample(frac=0.1)
logger.info("loaded signatures training data: {} rows {} columns", train_df.shape[0], train_df.shape[1])
test_df = df.drop(train_df.index)
logger.info("loaded signatures data: {} rows {} columns", test_df.shape[0], test_df.shape[1])

2024-06-14 18:59:31.500 | INFO     | __main__:<module>:2 - loaded signatures training data: 937 rows 3 columns
2024-06-14 18:59:31.504 | INFO     | __main__:<module>:4 - loaded signatures data: 8433 rows 3 columns


In [73]:
cluster_table: dict[int, list[str]] = {}
for cluster in df.cluster.unique():
    # cluster_table[cluster] = []
    cluster_df = train_df[df.cluster==cluster]
    cluster_table[cluster] = cluster_df.signature
assert len(cluster_table)

len(cluster_table[0])

/tmp/ipykernel_2972492/2082437146.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cluster_df = train_df[df.cluster==cluster]


115

In [74]:
import openai
import dotenv
import os
from openai import OpenAI 

dotenv.load_dotenv("../.env")

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

def get_embeddings(texts):
    client = openai.OpenAI()  # Create an OpenAI client
    model = "text-embedding-ada-002"  # Use the latest embedding model
    response = client.embeddings.create(
        input=texts,
        model=model
    )
    # embeddings = [data['embedding'] for data in response['data']]
    return response.model_dump()['data'] # type: ignore 

texts = cluster_table[0]
logger.info("processing {} inputs")
embeddings = get_embeddings(texts)
logger.info("obtained {} embeddings", len(embeddings))

2024-06-14 18:59:43.068 | INFO     | __main__:<module>:23 - processing 115 inputs


2024-06-14 18:59:43.926 | INFO     | __main__:<module>:25 - obtained 115 embeddings


In [75]:
embeds = [e['embedding'] for e in embeddings]
logger.info("extracted {} embeddings", len(embeds))

2024-06-14 18:59:48.463 | INFO     | __main__:<module>:2 - extracted 115 embeddings


In [76]:
train_df.head()

,cell_id,cluster,signature
4991,AGTAATGCACGTAATT,5,PDE4D PRKCB FCHSD2 HERC1 FCRL1 RALGPS2 REL PEL...
4325,TCTAAGGGTTTAACGG,4,ITGA4 CCND3 SMCHD1 EPS15 JAK1 EDEM3 UGGT1 AC07...
8808,AGTAACCTCCCTCGCA,15,SPTBN1 BACH2 EBF1 MME PTMA SSBP2 TCF4 TOP1
5931,TTAACCAAGGGTTAGA,7,FYN RUNX1 FAF1 CEMIP2 GPD2 CYTH1 RGS1 DENND1B ...
7835,GATTAAGCATGAATCT,11,ORC2 JMJD1C DIAPH3 ENSA ERVMER61-1 RIF1 SEPTIN...


In [89]:
import chromadb
import shutil
DB_PATH=Path("../data/testdb")
if DB_PATH.exists():
    shutil.rmtree(DB_PATH)
db_client = chromadb.PersistentClient(path="../data/testdb")
db_client.count_collections()

0

In [91]:
from chromadb.utils import embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="text-embedding-ada-002")
collection = db_client.create_collection(
    name="cell_embeddings",
    embedding_function=openai_ef) # type:ignore

OperationalError: attempt to write a readonly database